In [1]:
from src.parsers import mimic, hirid
from src.modeling import discovery, plots, querier, plots_v2 as plots_module
from src.utils import constants

In [2]:
import sys
# import win32com.client
import os
import pandas as pd

def setup_io_config(root_path):
    """
    Input - Output config. Add dataset paths
    :root_path -> Repo path which contains 'data' and 'res' folders
    """

    # MIMIC
    is_shortcut = True if "data.lnk" in os.listdir(root_path) else False 
    
    if (is_shortcut):
        path_shortcut =  os.path.join(root_path, "data.lnk")
        shell = win32com.client.Dispatch("WScript.Shell")
        mimic_data = shell.CreateShortCut(path_shortcut).Targetpath
    else:
        mimic_data = os.path.join(f"{root_path}", "data") 
    mimic_path = os.path.join(f"{root_path}", "results")

    # HIRID
    hirid_data = f'{root_path}/data/hirid-a-high-time-resolution-icu-dataset-1.1.1/raw_stage/'
    hirid_path = f'{root_path}/data/hirid-a-high-time-resolution-icu-dataset-1.1.1'
    
    return mimic_data, mimic_path, hirid_data, hirid_path

def setup_stratification_config():
    gender="MF"
    age_b=40
    age_a=80 
    ethnicity="WHITE" 
    lab_mapping= constants.LAB_MAPPING
    before_windows = [(0,6), (6,12)]
    after_windows = [(0,4), (4,8), (8,12)]
    return gender, age_a, age_b, ethnicity, lab_mapping, before_windows, after_windows

In [3]:
# IO Config
# root_path ="C:\\Users\\danco\\My Drive\\Master\\Datasets\\MIMIC iii"
root_path = "/Users/pavan/Library/CloudStorage/GoogleDrive-f20190038@hyderabad.bits-pilani.ac.in/My Drive/TAU/Code/DrugLab"
data, res, raw_path, res_path = setup_io_config(root_path=root_path)

# Stratification Config
gender, age_a, age_b, ethnicity, lab_mapping, before_windows, after_windows = setup_stratification_config()

In [4]:
med_lab_pairs = pd.read_csv("temp_med_lab_pairs.csv")

In [5]:
plotter = plots_module.ClinicalPlotsAnalysis(data, res, med_lab_pairs, plot_dir="plots", gender="MF", age_b=0, age_a=100, ethnicity="WHITE", lab_mapping=None)

In [6]:
#params
med_name = 'Insulin - Regular'
lab_name = 'Glucose'
start_time_before = 4 #hours
start_time_after = 8 #hours
min_interval_between_tests = 1 #hour
min_human = 20
max_human = 1000

In [7]:
plotter.plot_med_lab(
    med_name, 
    lab_name, 
    start_time_before=start_time_before, 
    start_time_after=start_time_after, 
    min_interval_between_tests=min_interval_between_tests, 
    min_human=min_human, max_human=max_human
)

<Figure size 1800x900 with 0 Axes>

In [8]:
plotter.plot_reg(
    med_name, 
    lab_name, 
    start_time_before=start_time_before, 
    start_time_after=start_time_after, 
    min_interval_between_tests=min_interval_between_tests, 
    min_human=min_human, max_human=max_human
)

<Figure size 600x600 with 0 Axes>

In [9]:
plotter.plot_all(
    start_time_before=start_time_before, 
    start_time_after=start_time_after, 
    min_interval_between_tests=min_interval_between_tests, 
    min_human=min_human, max_human=max_human
)

No Data for ACD-A Citrate (1000ml)<>Albumin pair
No Data for ACD-A Citrate (1000ml)<>Albumin pair
No Data for ACD-A Citrate (1000ml)<>Amylase pair
No Data for ACD-A Citrate (1000ml)<>Amylase pair
No Data for ACD-A Citrate (1000ml)<>Bilirubin pair
No Data for ACD-A Citrate (1000ml)<>Bilirubin pair
No Data for ACD-A Citrate (1000ml)<>Calcium pair
No Data for ACD-A Citrate (1000ml)<>Calcium pair
No Data for ACD-A Citrate (1000ml)<>Calcium ionized pair
No Data for ACD-A Citrate (1000ml)<>Calcium ionized pair
No Data for ACD-A Citrate (1000ml)<>Creatinine pair
No Data for ACD-A Citrate (1000ml)<>Creatinine pair
No Data for ACD-A Citrate (1000ml)<>Hemoglobin pair
No Data for ACD-A Citrate (1000ml)<>Hemoglobin pair
No Data for ACD-A Citrate (1000ml)<>Lactic acid pair
No Data for ACD-A Citrate (1000ml)<>Lactic acid pair
No Data for ACD-A Citrate (1000ml)<>Lipase pair
No Data for ACD-A Citrate (1000ml)<>Lipase pair
No Data for ACD-A Citrate (1000ml)<>Magnesium pair
No Data for ACD-A Citrate (10

/Users/pavan/Desktop/TAU/DrugLab/venv/lib/python3.9/site-packages/seaborn/axisgrid.py:1690: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  f = plt.figure(figsize=(height, height))


No Data for Acetaminophen-IV<>Hemoglobin pair
No Data for Acetaminophen-IV<>Hemoglobin pair
No Data for Acetaminophen-IV<>Hemoglobin percent pair
No Data for Acetaminophen-IV<>Hemoglobin percent pair
No Data for Acetaminophen-IV<>Lactic acid pair
No Data for Acetaminophen-IV<>Lactic acid pair
No Data for Acetaminophen-IV<>Lipase pair
No Data for Acetaminophen-IV<>Lipase pair
No Data for Acetaminophen-IV<>Magnesium pair
No Data for Acetaminophen-IV<>Magnesium pair
No Data for Acetaminophen-IV<>Potassium pair
No Data for Acetaminophen-IV<>Potassium pair
No Data for Acetaminophen-IV<>Prothrombin time INR pair
No Data for Acetaminophen-IV<>Prothrombin time INR pair
No Data for Acetaminophen-IV<>Red blood cell pair
No Data for Acetaminophen-IV<>Red blood cell pair
No Data for Acetaminophen-IV<>Uric Acid pair
No Data for Acetaminophen-IV<>Uric Acid pair
No Data for Acetaminophen-IV<>Vitamin B12 pair
No Data for Acetaminophen-IV<>Vitamin B12 pair
No Data for Acyclovir<>Albumin pair
No Data fo

/Users/pavan/Desktop/TAU/DrugLab/venv/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


No Data for Boost Glucose Control (Full)<>Potassium pair
No Data for Boost Glucose Control (Full)<>Potassium pair
No Data for Boost Glucose Control (Full)<>Prothrombin time INR pair
No Data for Boost Glucose Control (Full)<>Prothrombin time INR pair
No Data for Boost Glucose Control (Full)<>Red blood cell pair
No Data for Boost Glucose Control (Full)<>Red blood cell pair
No Data for Boost Glucose Control (Full)<>Uric Acid, Urine pair
No Data for Boost Glucose Control (Full)<>Uric Acid, Urine pair
No Data for Calcium Gluconate<>Albumin pair
No Data for Calcium Gluconate<>Albumin pair
No Data for Calcium Gluconate<>Calcium pair
No Data for Calcium Gluconate<>Calcium pair
No Data for Calcium Gluconate<>Calcium ionized pair
No Data for Calcium Gluconate<>Calcium ionized pair
No Data for Calcium Gluconate<>Creatinine pair
No Data for Calcium Gluconate<>Creatinine pair
No Data for Calcium Gluconate<>Hemoglobin pair
No Data for Calcium Gluconate<>Hemoglobin pair
No Data for Calcium Gluconate<

...

<Figure size 1800x900 with 0 Axes>

<Figure size 600x600 with 0 Axes>

<Figure size 1800x900 with 0 Axes>

<Figure size 600x600 with 0 Axes>

<Figure size 1800x900 with 0 Axes>